In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/output/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()

In [ ]:
from numpy import array
from numpy import argmax
from pandas import DataFrame
from nltk.translate.bleu_score import corpus_bleu
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Embedding
from keras.layers.merge import add
from keras.layers import Dropout
from numpy import argmax
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

# Image Extraction using VGG: 

# extract features 

def features_extraction(path,valid_files):
	# loading the VGG pre-trained model
	model = VGG16()
	# re-structuring the model, removing last layer from the model
	model.layers.pop()
	model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
	print(model.summary())
	# extracting features from each photo
	features = dict()
	for word in listdir(path):
		# loading a single image from file
		file = path + '/' + word
		# if word == 'indian_wedding_Image_19.png':
		# print(word)
		if word not in valid_files:
			print(word)
			continue
		#print("found")
		img = load_img(file, target_size=(224, 224))
		# convering the image pixel to array
		img = img_to_array(img)
		# reshape
		img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
		# prepare the image for the VGG model
		img = preprocess_input(img)
		# get features
		feature = model.predict(img, verbose=0)
		# get image id
		img_id = word.split('.')[0]
		# store feature
		features[img_id] = feature
		#print('>%s' % word)
	return features
 
# extract features from all images
path = '../input/indian-images/dlcv_indian_images'

import csv
with open('../input/set-files/ic_captions_processed_part_2.csv', mode='r') as infile:
     reader = csv.reader(infile)
     valid_files = [row[0] for row in reader]

print(len(valid_files))
features = features_extraction(path,valid_files)
print('Extracted VGG Features: %d' % len(features))
# save to file
dump(features, open('india_2_VGG16features.pkl', 'wb'))

In [ ]:
def LoadDoc(file_name):
    file = open(file_name, 'r', encoding = "utf-8")
    text_data = file.read()
    file.close()
    return text_data

In [ ]:
def LoadPhotoFeatures(file_name, dataset):
    AllFeatures = load(open(file_name, 'rb'))
    features = {k[:-4]: AllFeatures[k[:-4]] for k in dataset}
    return features

In [ ]:
def LoadCleanDescription(file_name):
  document = LoadDoc(file_name)
  descriptions = dict()
  i = 0
  for l in document.split('\n'):
    if len(l)<1:
      print(l,":",i)
      break
    i +=1
    if i == 1:
      continue
    tokens = l.split(',')
    imageId, imageDesc = tokens[0], tokens[1].split()
    if imageId not in descriptions:
        descriptions[imageId] = list()
    desc = 'startseq ' + ' '.join(imageDesc) + ' endseq'
    descriptions[imageId].append(desc)
  return descriptions

In [ ]:
def ToLines(desc):
    all_desc = list()
    for key in desc.keys():
        [all_desc.append(d) for d in desc[key]]
    return all_desc

In [ ]:
from transformers import AutoTokenizer

In [ ]:
def CreateTokenizer(desc):
    tokenizer = AutoTokenizer.from_pretrained('monsoon-nlp/hindi-bert')
    return tokenizer

In [ ]:
def maxLength(descs):
    lines = ToLines(descs)
    return max(len(d.split()) for d in lines)

In [ ]:
# define the captioning model
def DefineModel(vocabSize, max_length):
    inputs1 = Input(shape=(1000,))
    fe1 = Dropout(0.5, seed = 121)(inputs1)
    fe2 = Dense(128, activation='relu')(fe1)
    fe3 = RepeatVector(max_length)(fe2)
    
    # embedding
    inputs2 = Input(shape=(max_length,))
    emb2 = Embedding(vocab_size, 50, mask_zero=True)(inputs2)
    emb2a = Dropout(0.5, seed = 121)(emb2)
    emb3 = LSTM(128, return_sequences=True)(emb2a)
    emb4 = LSTM(128, return_sequences=True)(emb3)
    emb5 = TimeDistributed(Dense(128, activation='relu'))(emb4)
    # merge inputs
    merged = add([fe3, emb5])
    # language model (decoder)
    lm1 = Dropout(0.5, seed = 121)(merged)
    lm2 = LSTM(500)(lm1)
    lm3 = Dense(500, activation='relu')(lm2)
    outputs = Dense(vocab_size, activation='softmax')(lm3)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer = 'adam' , metrics=['accuracy'])
    print(model.summary())
    plot_model(model, show_shapes=True, to_file='plot.png')
    return model

In [ ]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'True'

In [ ]:
def CreateSequence(tokenizer, max_length, desc, images, vocabSize):
    X1, X2, y = list(), list(), list()
    for key, desc_list in desc.items():
        #print(key)
        for desc_sent in desc_list:
#             print(datetime.now())
            #print(desc_sent)
#             sequence = tokenizer.texts_to_sequences([desc_sent])[0]
            sequence = tokenizer([desc_sent])['input_ids'][0]
            for i in range(1, len(sequence)):
                in_sequence, out_sequence = sequence[:i], sequence[i]
                in_sequence = pad_sequences([in_sequence], maxlen=max_length)[0]
                out_sequence = to_categorical([out_sequence], num_classes=vocabSize)[0]
                X1.append(images[key[:-4]][0])
                X2.append(in_sequence)
                y.append(out_sequence)
                # break
            #break
        #break
    return array(X1), array(X2), array(y)

In [ ]:
train_descriptions = LoadCleanDescription('/kaggle/input/set-files/train_file.tsv')
# test_descriptions = LoadCleanDescription('/kaggle/input/setfiles/test_file.tsv')
valid_descriptions = LoadCleanDescription('/kaggle/input/set-files/valid_file.tsv')
print('Descriptions: train=%d, valid=%d' % (len(train_descriptions), len(valid_descriptions)))

In [ ]:
train_features = LoadPhotoFeatures('/kaggle/input/set-files/ReNet50features.pkl', train_descriptions)
# test_features = LoadPhotoFeatures('/kaggle/input/setfiles/ReNet50features.pkl', test_descriptions)
valid_features = LoadPhotoFeatures('/kaggle/input/set-files/ReNet50features.pkl', valid_descriptions)

In [ ]:
print('Photos: train=%d, valid=%d' % (len(train_features), len(valid_features)))

In [ ]:
tokenizer = CreateTokenizer(train_descriptions)
vocab_size = len(tokenizer.vocab)+1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = maxLength(train_descriptions)
print('Description Length: %d' % max_length)

In [ ]:
model_name = 'Ex18'
verbose = 2
n_epochs = 30
n_photos_per_update = 2
# n_batches_per_epoch = int(len(train_descriptions.keys()) / n_photos_per_update)
n_repeats = 5

# for i in range(n_repeats):
model = DefineModel(vocab_size, max_length)
X1train, X2train, ytrain = CreateSequence(tokenizer, max_length, train_descriptions, train_features, vocab_size)
X1valid, X2valid, yvalid = CreateSequence(tokenizer, max_length, valid_descriptions, valid_features, vocab_size)
hist = model.fit([X1train, X2train], ytrain, epochs=10, verbose=2, validation_data=([X1test, X2test], ytest)) 
model.save("my_model_2")

In [ ]:
import numpy as np
import keras

class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,ImageFeatures,tokenizer,max_length,descriptions,vocabSize,batch_size=32):
        'Initialization'
        self.batch_size = batch_size
        self.ImageFeatures = ImageFeatures
        self.tokenizer = tokenizer
        self.maxLength = max_length
        self.descriptions = descriptions
        self.vocabSize = vocabSize
        self.keys = list(descriptions.keys())
#         self.X1,self.X2,self.Y = CreateSequence(self.tokenizer, self.maxLength, self.descriptions,self.ImageFeatures,self.vocabSize)

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.descriptions) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        print(datetime.now())
        batch_keys = self.keys[index*self.batch_size:(index+1)*self.batch_size]
        batch_descriptions = { your_key: self.descriptions[your_key] for your_key in batch_keys }
        batch_features = { your_key[:-4]: self.ImageFeatures[your_key[:-4]] for your_key in batch_keys }
        X1,X2,Y = CreateSequence(self.tokenizer, self.maxLength, batch_descriptions,batch_features,self.vocabSize)
        X_final = [X1, X2]
        Y_final = Y
        return X_final, Y_final
    
#     def CreateSequence(tokenizer, maxLength, desc, images, vocabSize):
#         X1, X2, y = list(), list(), list()
#         for key, desc_list in desc.items():
#             #print(key)
#             for desc_sent in desc_list:
#                 #print(desc_sent)
#                 sequence = tokenizer.texts_to_sequences([desc_sent])[0]
#                 for i in range(1, len(sequence)):
#                     in_sequence, out_sequence = sequence[:i], sequence[i]
#                     in_sequence = pad_sequences([in_sequence], maxlen=maxLength)[0]
#                     out_sequence = to_categorical([out_sequence], num_classes=vocabSize)[0]
#                     X1.append(images[key[:-4]][0])
#                     X2.append(in_sequence)
#                     y.append(out_sequence)
#                     # break
#                 #break
#             #break
#         return array(X1), array(X2), array(y)

In [ ]:
training_generator = DataGenerator(train_features,tokenizer,max_length,train_descriptions,vocab_size)

In [ ]:
model = DefineModel(vocab_size, max_length)
model.fit_generator(generator=training_generator,
                    use_multiprocessing=True,
                    epochs=20, verbose=2,
                    workers=6)

In [ ]:
from datetime import datetime

In [ ]:
model.save("my_model_5_captions")

In [ ]:
# generate a description for an image
def GenerateDesc(model, tokenizer, image, max_length):
        start_text = 'startseq'
        for i in range(max_length):
                seq = tokenizer.texts_to_sequences([start_text])[0]
                seq = pad_sequences([seq], maxlen=max_length)
                yword = model.predict([image,seq], verbose=0,workers=6, use_multiprocessing=True)
                yword = argmax(yword)
                word = WordForId(yword, tokenizer)
                if word is None:
                        break
                start_text += ' ' + word
                if word == 'endseq':
                        break
        return start_text

# evaluate the skill of the model
def EvaluateModel(model, descs, images, tokenizer, max_length):
        y, yhat = list(), list()
        i = 0
        for key, desc_list in descs.items():
                print("key",key)
                yword = GenerateDesc(model, tokenizer, images[key[:-4]], max_length)
                references = [d.split() for d in desc_list]
                print("yword",yword)
                print("references",references)
                y.append(references)
                yhat.append(yword.split())
                if i!=0 and i%20==0:
                    print('BLEU-1: %f' % corpus_bleu(y, yhat, weights=(1.0, 0, 0, 0)))
                    break
                i+=1
        # calculate BLEU score
        print('BLEU-1: %f' % corpus_bleu(y, yhat, weights=(1.0, 0, 0, 0)))
        print('BLEU-2: %f' % corpus_bleu(y, yhat, weights=(0.5, 0.5, 0, 0)))
        print('BLEU-3: %f' % corpus_bleu(y, yhat, weights=(0.3, 0.3, 0.3, 0)))
        print('BLEU-4: %f' % corpus_bleu(y, yhat, weights=(0.25, 0.25, 0.25, 0.25)))

In [ ]:
test_descriptions = LoadCleanDescription('/kaggle/input/set-files/test_file.tsv')
# photo features
test_features = LoadPhotoFeatures('/kaggle/input/set-files/ReNet50features.pkl', test_descriptions)
# descriptions

In [ ]:
def WordForId(integer, tokenizer):
	for w, i in tokenizer.word_index.items():
		if i == integer:
			return w
	return None

In [ ]:
def DefineModel(vocabSize, maxLength):
    inputs1 = Input(shape=(1000,))
    fe1 = Dropout(0.5, seed = 121)(inputs1)
    fe2 = Dense(128, activation='relu')(fe1)
    fe3 = RepeatVector(max_length)(fe2)
    
    # embedding
    inputs2 = Input(shape=(max_length,))
    emb2 = Embedding(vocab_size, 50, mask_zero=True)(inputs2)
    emb2a = Dropout(0.5, seed = 121)(emb2)
    emb3 = LSTM(128, return_sequences=True)(emb2a)
    emb4 = LSTM(128, return_sequences=True)(emb3)
    emb5 = TimeDistributed(Dense(128, activation='relu'))(emb4)
    # merge inputs
    merged = add([fe3, emb5])
    # language model (decoder)
    lm1 = Dropout(0.5, seed = 121)(merged)
    lm2 = LSTM(500)(lm1)
    lm3 = Dense(500, activation='relu')(lm2)
    outputs = Dense(vocab_size, activation='softmax')(lm3)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer = 'adam' , metrics=['accuracy'])
    print(model.summary())
    plot_model(model, show_shapes=True, to_file='plot.png')
    return model

In [ ]:
test_descriptions = LoadCleanDescription('/kaggle/input/set-files/test_file.tsv')
# photo features
test_features = LoadPhotoFeatures('../input/set-files/VGG16features (2).pkl', test_descriptions)
# descriptions

In [ ]:
model = load_model("/kaggle/input/set-files/vgg16_5_captions")
# evaluate model


In [ ]:
EvaluateModel(model, test_descriptions, test_features, tokenizer, max_length)

In [ ]:
EvaluateModel(model, test_descriptions, test_features, tokenizer, max_length)
# resnet

In [ ]:
!zip -r model_resnet_5_captions_20_epochs.zip my_model_5_captions

In [ ]:
EvaluateModel(model, test_descriptions, test_features, tokenizer, max_length)

In [ ]:
test_descriptions = LoadCleanDescription('/kaggle/input/set-files/test_file.tsv')
# photo features
test_features = LoadPhotoFeatures('/kaggle/input/set-files/ReNet50features.pkl', test_descriptions)
# descriptions

In [ ]:
model = load_model("/kaggle/input/set-files/resnet_5_captions")

In [ ]:

EvaluateModel(model, test_descriptions, test_features, tokenizer, max_length)
# Resnet full eval



In [ ]:
import time
validation_generator = DataGenerator(valid_features,tokenizer,max_length,valid_descriptions,vocab_size)
filename = 'model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
start = time.time()
model = DefineModel(vocab_size, max_length)
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    epochs=20, verbose=2,
                    workers=6)
end = time.time()
print("TIME TOOK {:3.2f}MIN".format((end - start )/60))
    
#Plot Graph

for label in ["loss","val_loss"]:
    plt.plot(hist.history[label],label=label)
plt.legend()
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()
plt.savefig('ModelPerformance.png')

In [ ]:
tokenizer(['विकिपीडिया सभी विषयों पर प्रामाणिक और उपयोग, परिवर्तन व पुनर्वितरण के लिए स्वतन्त्र ज्ञानकोश बनाने का एक बहुभाषीय प्रकल्प है।'])['input_ids']

In [ ]:
from indicnlp.tokenize import sentence_tokenize

In [ ]:
def CreateTokenizer():
    tokenizer = AutoTokenizer.from_pretrained('facebook/mbart-large-50-many-to-many-mmt')
    return tokenizer

In [ ]:
tokenizer = CreateTokenizer()

In [ ]:
tokenizer(['कहड में चाई और साथ में ब्रेड का टुकड़ा ','विकिपीडिया सभी विषयों पर प्रामाणिक और उपयोग, परिवर्तन व पुनर्वितरण के लिए स्वतन्त्र ज्ञानकोश बनाने का एक बहुभाषीय प्रकल्प है।'])['input_ids']

In [ ]:
!nvidia-smi